# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing Turi Create functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in Turi Create functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Fire up Turi Create

In [2]:
import turicreate

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [3]:
sales = turicreate.SFrame('kc_house_data.csv')

Finished parsing file /home/astha/kc_house_data.csv

Parsing completed. Parsed 100 lines in 0.347183 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,float,int,float,int,int,float,int,int,int,int,int,int,int,int,int,float,float,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /home/astha/kc_house_data.csv

Parsing completed. Parsed 21613 lines in 0.143666 secs.

In [20]:
sales

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view
7129300520,20141013T000000,221900.0,3,1.0,1180,5650,1.0,0,0
6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0
5631500400,20150225T000000,180000.0,2,1.0,770,10000,1.0,0,0
2487200875,20141209T000000,604000.0,4,3.0,1960,5000,1.0,0,0
1954400510,20150218T000000,510000.0,3,2.0,1680,8080,1.0,0,0
7237550310,20140512T000000,1225000.0,4,4.5,5420,101930,1.0,0,0
1321400060,20140627T000000,257500.0,3,2.25,1715,6819,2.0,0,0
2008000270,20150115T000000,291850.0,3,1.5,1060,9711,1.0,0,0
2414600126,20150415T000000,229500.0,3,1.0,1780,7470,1.0,0,0
3793500160,20150312T000000,323000.0,3,2.5,1890,6560,2.0,0,0


# Load training and testing data


In [5]:
train_data = turicreate.SFrame('kc_house_train_data.csv')

Finished parsing file /home/astha/kc_house_train_data.csv

Parsing completed. Parsed 100 lines in 0.165173 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,float,int,float,int,int,float,int,int,int,int,int,int,int,int,int,float,float,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /home/astha/kc_house_train_data.csv

Parsing completed. Parsed 17384 lines in 0.120893 secs.

In [4]:
test_data = turicreate.SFrame('kc_house_test_data.csv')

Finished parsing file /home/astha/kc_house_test_data.csv

Parsing completed. Parsed 100 lines in 0.073654 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,float,int,float,int,int,float,int,int,int,int,int,int,int,int,int,float,float,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /home/astha/kc_house_test_data.csv

Parsing completed. Parsed 4229 lines in 0.066208 secs.

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [6]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
example_model = turicreate.linear_regression.create(train_data, target = 'price', features = example_features, 
                                                    validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 3

Number of unpacked features : 3

...

| Iteration | Passes   | Elapsed Time | Training Max Error | Training Root-Mean-Square Error |

+-----------+----------+--------------+--------------------+---------------------------------+

| 1         | 2        | 1.022395     | 4146407.590959     | 258679.804259                   |

+-----------+----------+--------------+--------------------+---------------------------------+

SUCCESS: Optimal solution found.

Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [8]:
example_weight_summary = example_model.coefficients
print (example_weight_summary)

+-------------+-------+---------------------+--------------------+
|     name    | index |        value        |       stderr       |
+-------------+-------+---------------------+--------------------+
| (intercept) |  None |  87910.06920678669  | 7873.338136755249  |
| sqft_living |  None |  315.4034411181024  | 3.4557003229375387 |
|   bedrooms  |  None | -65080.215441625194 | 2717.4568521267206 |
|  bathrooms  |  None |  6944.020876313873  | 3923.1149281297144 |
+-------------+-------+---------------------+--------------------+
[4 rows x 4 columns]



# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing Turi Create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [10]:
example_predictions = example_model.predict(train_data)
print (example_predictions[0])

271789.50427758583


# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [14]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    predictions = model.predict(data)

    # Then compute the residuals/errors
    residuals = predictions - outcome

    # Then square and add them up
    RSS = (residuals**2).sum()

    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [15]:
rss_example_train = get_residual_sum_of_squares(example_model, test_data, test_data['price'])
print (rss_example_train) 

273761534465419.88


# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [16]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [17]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2) 

In [18]:
# create the remaining 3 features in both TEST and TRAIN data
train_data['bed_bath_rooms'] = train_data['bedrooms']*train_data['bathrooms']
test_data['bed_bath_rooms'] = test_data['bedrooms']*test_data['bathrooms']

In [19]:
train_data['log_sqft_living'] = train_data['sqft_living'].apply(lambda x: log(x))
test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x: log(x))

In [21]:
train_data['lat_plus_long'] = train_data['lat'] + train_data['long']
test_data['lat_plus_long'] = test_data['lat'] + test_data['long']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

In [44]:
train_data

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view
7129300520,20141013T000000,221900.0,3,1.0,1180,5650,1.0,0,0
6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0
5631500400,20150225T000000,180000.0,2,1.0,770,10000,1.0,0,0
2487200875,20141209T000000,604000.0,4,3.0,1960,5000,1.0,0,0
1954400510,20150218T000000,510000.0,3,2.0,1680,8080,1.0,0,0
7237550310,20140512T000000,1225000.0,4,4.5,5420,101930,1.0,0,0
1321400060,20140627T000000,257500.0,3,2.25,1715,6819,2.0,0,0
2008000270,20150115T000000,291850.0,3,1.5,1060,9711,1.0,0,0
2414600126,20150415T000000,229500.0,3,1.0,1780,7470,1.0,0,0
3793500160,20150312T000000,323000.0,3,2.5,1890,6560,2.0,0,0


**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

In [28]:
print(train_data['bedrooms_squared'].mean())
print(train_data['bed_bath_rooms'].mean())
print(train_data['log_sqft_living'].mean())
print(train_data['lat_plus_long'].mean())

12.174240681086044
7.496591693511277
7.550349317685001
-74.65396803382424


# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [29]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using turicreate.linear_regression.create() and look at the value of the weights/coefficients:

In [30]:
# Learn the three models: (don't forget to set validation_set = None)
model_1 = turicreate.linear_regression.create(train_data, target = 'price', features = model_1_features, 
                                                    validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 5

Number of unpacked features : 5

...

+-----------+----------+--------------+--------------------+---------------------------------+

| 1         | 2        | 0.039952     | 4074792.266563     | 236378.065704                   |

| 2         | 3        | 0.098964     | 4074792.266501     | 236378.065703                   |

+-----------+----------+--------------+--------------------+---------------------------------+

SUCCESS: Optimal solution found.

In [31]:
model_2 = turicreate.linear_regression.create(train_data, target = 'price', features = model_2_features, 
                                                    validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 6

Number of unpacked features : 6

...

+-----------+----------+--------------+--------------------+---------------------------------+

| 1         | 2        | 0.065316     | 4014053.068791     | 235190.383244                   |

| 2         | 3        | 0.104196     | 4014053.068757     | 235190.383243                   |

+-----------+----------+--------------+--------------------+---------------------------------+

SUCCESS: Optimal solution found.

In [32]:
model_3 = turicreate.linear_regression.create(train_data, target = 'price', features = model_3_features, 
                                                    validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 9

Number of unpacked features : 9

...

+-----------+----------+--------------+--------------------+---------------------------------+

| 1         | 2        | 0.052632     | 3193116.484483     | 228199.723039                   |

| 2         | 3        | 0.100293     | 3193116.484452     | 228199.723038                   |

+-----------+----------+--------------+--------------------+---------------------------------+

SUCCESS: Optimal solution found.

In [38]:
# Examine/extract each model's coefficients:
print('model_1 coefficients: \n',model_1.coefficients)


model_1 coefficients: 
 +-------------+-------+--------------------+--------------------+
|     name    | index |       value        |       stderr       |
+-------------+-------+--------------------+--------------------+
| (intercept) |  None | -56141852.46435269 | 1649928.3580122495 |
| sqft_living |  None | 310.2635067230146  | 3.1888206232738048 |
|   bedrooms  |  None | -59577.60521214531 | 2487.2745584621066 |
|  bathrooms  |  None | 13812.406875960849 | 3593.5344057244292 |
|     lat     |  None | 629863.1199344355  | 13120.688670221312 |
|     long    |  None | -214800.9523004764 | 13283.820389035558 |
+-------------+-------+--------------------+--------------------+
[6 rows x 4 columns]



In [39]:
print('model_2 coefficients: \n',model_2.coefficients)

model_2 coefficients: 
 +----------------+-------+---------------------+--------------------+
|      name      | index |        value        |       stderr       |
+----------------+-------+---------------------+--------------------+
|  (intercept)   |  None |  -54412036.39657792 | 1650346.0975324612 |
|  sqft_living   |  None |  304.44950523902344 | 3.2021651828164113 |
|    bedrooms    |  None | -116366.74281849852 | 4805.533985505644  |
|   bathrooms    |  None |  -77972.1056409074  | 7565.034160780051  |
|      lat       |  None |  625431.0420620056  | 13058.33083654401  |
|      long      |  None | -203970.82493612493 | 13267.649041637562 |
| bed_bath_rooms |  None |  26961.730297756807 | 1956.358993667354  |
+----------------+-------+---------------------+--------------------+
[7 rows x 4 columns]



In [40]:
print('model_3 coefficients: \n',model_3.coefficients)

model_3 coefficients: 
 +------------------+-------+---------------------+--------------------+
|       name       | index |        value        |       stderr       |
+------------------+-------+---------------------+--------------------+
|   (intercept)    |  None |  -52975504.2139663  | 1615141.2915367081 |
|   sqft_living    |  None |   529.193272250334  | 7.699122573047254  |
|     bedrooms     |  None |  28944.217163037203 | 9395.700864455763  |
|    bathrooms     |  None |   65659.9030562364  | 10795.320046410321 |
|       lat        |  None |  704763.1651200312  | 2126816724.9806376 |
|       long       |  None | -137781.47139257815 | 2126816724.116207  |
|  bed_bath_rooms  |  None |  -8477.927639033875 | 2858.9496551764973 |
| bedrooms_squared |  None |  -6072.037638551657 | 1494.9671654266658 |
| log_sqft_living  |  None |  -563459.7580526089 | 17567.817600469152 |
|  lat_plus_long   |  None |  -83220.68126129049 |  2126816724.84827  |
+------------------+-------+------------

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

Think about what this means.

Answer: Coefficient/weight for 'bathrooms' in model 1 is 13812.406875960849 while in model 2 it's -77972.1056409074.

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [42]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
print('RSS for model_1 training_data: ', get_residual_sum_of_squares(model_1, train_data, train_data['price']))
print('RSS for model_2 training_data: ', get_residual_sum_of_squares(model_2, train_data, train_data['price']))
print('RSS for model_3 training_data: ', get_residual_sum_of_squares(model_3, train_data, train_data['price']))

RSS for model_1 training_data:  971323871612284.9
RSS for model_2 training_data:  961587552579545.2
RSS for model_3 training_data:  905273774729645.4


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

model_3 has lowest RSS value.

Now compute the RSS on on TEST data for each of the three models.

In [43]:
# Compute the RSS on TESTING data for each of the three models and record the values:
print('RSS for model_1 test_data: ', get_residual_sum_of_squares(model_1, test_data, test_data['price']))
print('RSS for model_2 test_data: ', get_residual_sum_of_squares(model_2, test_data, test_data['price']))
print('RSS for model_3 test_data: ', get_residual_sum_of_squares(model_3, test_data, test_data['price']))

RSS for model_1 test_data:  226566318597969.6
RSS for model_2 test_data:  224366990581798.62
RSS for model_3 test_data:  251824565047037.2


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected? Think about the features that were added to each model from the previous.

model_2 has the lowest RSS value